In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    query: str
    context: list
    answer: str

graph_builder = StateGraph(AgentState)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0
)

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

generate_prompt = hub.pull("rlm/rag-prompt")


def web_generate(state: AgentState) -> AgentState:
    context = state["context"]
    query = state["query"]
    rag_chain = generate_prompt | llm | StrOutputParser()
    response = rag_chain.invoke({"question": query, "context": context})
    return {"answer": response}

In [ ]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True
)

def web_search(state:AgentState):
    query = state["query"]
    results = tavily_search_tool.invoke(query)
    return {"context": results}



In [ ]:
from langchain_core.output_parsers import StrOutputParser
basic_llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0
)

def basic_generate(state: AgentState):
    query = state["query"]
    basic_llm_chain = basic_llm | StrOutputParser()
    llm_response = basic_llm_chain.invoke(query)
    return {"answer": llm_response}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal


class Route(BaseModel):
    target: Literal['vector_store','llm','web_search'] = Field(
        description = "The target for the query to answer"
    )

router_system_prompt = """
You are an export at routing a user's question to 'vector_store', 'llm' or 'web_search'.
'vector_store' contains information about income tax up to 2025.
if you think the question is simple enought use 'llm'
if you think you need to search the web to answer the question use 'web_search'
"""

router_prompt = ChatPromptTemplate.from_messages([
    ('system', router_system_prompt),
    ('user','{query}')
])

router_llm = ChatOllama(model="deepseek-r1:1.5b")
structured_router_llm= router_llm.with_structured_output(Route)



def router(state: AgentState):
    query = state["query"]
    router_chain = router_prompt | structured_router_llm
    route = router_chain.invoke({"query": query})
    return route


In [ ]:
from income_tax_graph import graph as income_tax_graph

graph_builder.add_node('income_tax_agent', income_tax_graph)
graph_builder.add_node('web_generate', web_generate)
graph_builder.add_node('web_search', web_search)
graph_builder.add_node('basic_generate', basic_generate)


In [ ]:
from langgraph.graph import START, END

graph_builder.add_conditional_edges(
    START,
    router,
    {
        'vector_store': 'income_tax_agent',
        'llm': 'basic_generate',
        'web_search': 'web_search'
    }
)

graph_builder.add_edge('web_search', 'web_generate')
graph_builder.add_edge('web_generate', END)
graph_builder.add_edge('basic_generate', END)
graph_builder.add_edge('income_tax_agent', END)


In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))